## Imports

In [63]:
import numpy as np
import pandas as pd
import random
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix

In [11]:
# importar base 1
df_base1 = pd.read_csv('/content/base01.csv')

# transformando label em inteiros
df_base1['21'] = df_base1['21'].astype('category')
df_base1['21'] = df_base1['21'].cat.codes

# separar classes
class0_df = df_base1[df_base1['21'] == 0]
class1_df = df_base1[df_base1['21'] == 1]

print('Classe 0: ', class0_df.shape, '\nClasse 1: ', class1_df.shape)

Classe 0:  (415, 23) 
Classe 1:  (107, 23)


In [12]:
# importar base 2
df_base2 = pd.read_csv('/content/base02.csv')

# transformande label em inteiros
df_base2['21'] = df_base2['21'].astype('category')
df_base2['21'] = df_base2['21'].cat.codes

# separar classes
class0_df2 = df_base2[df_base2['21'] == 0]
class1_df2 = df_base2[df_base2['21'] == 1]

print('Classe 0: ', class0_df2.shape, '\nClasse 1: ', class1_df2.shape)

Classe 0:  (1783, 23) 
Classe 1:  (326, 23)


## Algoritmo

In [38]:
def train(array_input):
  # selecionar um pivô aleatorio
  n = random.randrange(0, len(array_input)-1)
  pivo = array_input[n]
  # cria fila com o pivo
  q = []
  q.append(pivo)
  # lista com pontos da borda
  b = []
  # rodar busca em largura usando a distância euclidiana como limiar
  borda = busca_largura(array_input, q, b)
  # retornar aproximação da borda
  return borda, pivo  

In [58]:
def busca_largura(array_input, q, b):
  # remove primeiro elemento da fila = pivo
  pivo = q[0]
  q.pop(0)
  
  # cria array temporario sem o pivo
  array_temp = np.copy(array_input)
  for n in range(len(array_input)):
    if (array_input[n]==pivo).all():
      array_temp = np.delete(array_temp, n, 0)
  # copia array temporario
  array_aux = np.copy(array_temp)
  #conta vizinhos
  vizinhos = 0
  
  # para cada item em array_temp calcular distancia euclidiana para pivo
  for i in range(len(array_temp)):
    de = np.linalg.norm(pivo-array_temp[i])
    # se a distancia <= 5 coloca na fila
    if de <= 5:
      vizinhos = vizinhos + 1
      q.append(array_temp[i])
      array_aux = np.delete(array_aux, i, 0)

  # se o pivo nao possui vizinho entao ele esta na borda
  if vizinhos == 0:
    b.append(pivo)

  #se nao ha mais elementos na fila retorna
  if len(q) == 0:
    return b
  else:
    # do contrario chama novamente a funcao com fila e array com pontos restantes
    busca_largura(array_aux, q, b)

In [60]:
def predict(pivo, borda, array_teste):
  
  labels = []

  for i in array_teste:
    # para cada item no array de teste encontrar item da borda mais proximo
    valor_inicial = np.linalg.norm(i-borda[0])
    posicao = 0

    for k in range(1,len(borda)):
      valor_atual = np.linalg.norm(i-borda[k])
      if valor_atual < valor_inicial:
        valor_inicial = valor_atual
        posicao = k
    
    # calcula distancia euclidiana do pivo ao novo ponto
    pivo_a_np = np.linalg.norm(pivo-i)

    # calcula distancia euclidiana do pivo ao ponto da borda
    pivo_a_borda = np.linalg.norm(pivo-borda[posicao])

    # se a distancia do pivo para o novo ponto for menor que a distancia do
    # pivo para a borda, entao o novo ponto deve ser negativo. caso contrario
    # o novo ponto deve ser positivo.
    if pivo_a_np < pivo_a_borda:
      np_label = 0
    else:
      np_label = 1

    labels.append(np_label)

  # retornar lista com labels
  return labels

## 30% dos dados

In [65]:
# separar labels
X0 = class0_df.iloc[:, :-1].values
y0 = class0_df.iloc[:, -1].values

# selecionar 30% dos dados da base 1 (156 instancias) para treino
X_train = X[:156]

# treinar sobre esses dados
b, p = train(X_train)

# juntar restante das instancias da classe 0 com as instancias da classe 1
X1 = class1_df.iloc[:, :-1].values
y1 = class1_df.iloc[:, -1].values

X0_test = X0[156:]
y0_test = y0[156:]
X_test = np.concatenate((X0_test, X1), axis=0)
y_test = np.concatenate((y0_test, y1), axis=0)

# testar sobre o resto da base
y_pred = predict(p, b, X_test)

# calcular f-score, true positive, true negative
f_score = f1_score(y_test, y_pred, average='weighted')
print('F1-measure: ', f_score)
conf_matrix = confusion_matrix(y_test, y_pred)
true_pos = conf_matrix[1][1]
false_pos = conf_matrix[0][1]
print('True Positives: ', true_pos)
print('False Positives: ', false_pos)

F1-measure:  0.1322681639113206
True Positives:  107
False Positives:  259


In [73]:
# separar labels
X0 = class0_df2.iloc[:, :-1].values
y0 = class0_df2.iloc[:, -1].values

# selecionar 30% dos dados da base 1 (632 instancias) para treino
X_train = X[:632]

# treinar sobre esses dados
b, p = train(X_train)

# juntar restante das instancias da classe 0 com as instancias da classe 1
X1 = class1_df2.iloc[:, :-1].values
y1 = class1_df2.iloc[:, -1].values

X0_test = X0[632:]
y0_test = y0[632:]
X_test = np.concatenate((X0_test, X1), axis=0)
y_test = np.concatenate((y0_test, y1), axis=0)

# testar sobre o resto da base
y_pred = predict(p, b, X_test)

# calcular f-score, true positive, true negative
f_score = f1_score(y_test, y_pred, average='weighted')
print('F1-measure: ', f_score)
conf_matrix = confusion_matrix(y_test, y_pred)
true_pos = conf_matrix[1][1]
false_pos = conf_matrix[0][1]
print('True Positives: ', true_pos)
print('False Positives: ', false_pos)

F1-measure:  0.079815818892082
True Positives:  326
False Positives:  1151


## 40% dos dados

In [67]:
# separar labels
X0 = class0_df.iloc[:, :-1].values
y0 = class0_df.iloc[:, -1].values

# selecionar 40% dos dados da base 1 (208 instancias) para treino
X_train = X[:208]

# treinar sobre esses dados
b, p = train(X_train)

# juntar restante das instancias da classe 0 com as instancias da classe 1
X1 = class1_df.iloc[:, :-1].values
y1 = class1_df.iloc[:, -1].values

X0_test = X0[208:]
y0_test = y0[208:]
X_test = np.concatenate((X0_test, X1), axis=0)
y_test = np.concatenate((y0_test, y1), axis=0)

# testar sobre o resto da base
y_pred = predict(p, b, X_test)

# calcular f-score, true positive, true negative
f_score = f1_score(y_test, y_pred, average='weighted')
print('F1-measure: ', f_score)
conf_matrix = confusion_matrix(y_test, y_pred)
true_pos = conf_matrix[1][1]
false_pos = conf_matrix[0][1]
print('True Positives: ', true_pos)
print('False Positives: ', false_pos)

F1-measure:  0.17321512322798308
True Positives:  107
False Positives:  207


In [70]:
# separar labels
X0 = class0_df2.iloc[:, :-1].values
y0 = class0_df2.iloc[:, -1].values

# selecionar 40% dos dados da base 1 (843 instancias) para treino
X_train = X[:843]

# treinar sobre esses dados
b, p = train(X_train)

# juntar restante das instancias da classe 0 com as instancias da classe 1
X1 = class1_df2.iloc[:, :-1].values
y1 = class1_df2.iloc[:, -1].values

X0_test = X0[843:]
y0_test = y0[843:]
X_test = np.concatenate((X0_test, X1), axis=0)
y_test = np.concatenate((y0_test, y1), axis=0)

# testar sobre o resto da base
y_pred = predict(p, b, X_test)

# calcular f-score, true positive, true negative
f_score = f1_score(y_test, y_pred, average='weighted')
print('F1-measure: ', f_score)
conf_matrix = confusion_matrix(y_test, y_pred)
true_pos = conf_matrix[1][1]
false_pos = conf_matrix[0][1]
print('True Positives: ', true_pos)
print('False Positives: ', false_pos)

F1-measure:  0.10546016020068746
True Positives:  326
False Positives:  940


## 50% dos dados

In [68]:
# separar labels
X0 = class0_df.iloc[:, :-1].values
y0 = class0_df.iloc[:, -1].values

# selecionar 50% dos dados da base 1 (261 instancias) para treino
X_train = X[:261]

# treinar sobre esses dados
b, p = train(X_train)

# juntar restante das instancias da classe 0 com as instancias da classe 1
X1 = class1_df.iloc[:, :-1].values
y1 = class1_df.iloc[:, -1].values

X0_test = X0[261:]
y0_test = y0[261:]
X_test = np.concatenate((X0_test, X1), axis=0)
y_test = np.concatenate((y0_test, y1), axis=0)

# testar sobre o resto da base
y_pred = predict(p, b, X_test)

# calcular f-score, true positive, true negative
f_score = f1_score(y_test, y_pred, average='weighted')
print('F1-measure: ', f_score)
conf_matrix = confusion_matrix(y_test, y_pred)
true_pos = conf_matrix[1][1]
false_pos = conf_matrix[0][1]
print('True Positives: ', true_pos)
print('False Positives: ', false_pos)

F1-measure:  0.2384016325170748
True Positives:  107
False Positives:  154


In [71]:
# separar labels
X0 = class0_df2.iloc[:, :-1].values
y0 = class0_df2.iloc[:, -1].values

# selecionar 50% dos dados da base 1 (1054 instancias) para treino
X_train = X[:1054]

# treinar sobre esses dados
b, p = train(X_train)

# juntar restante das instancias da classe 0 com as instancias da classe 1
X1 = class1_df2.iloc[:, :-1].values
y1 = class1_df2.iloc[:, -1].values

X0_test = X0[1054:]
y0_test = y0[1054:]
X_test = np.concatenate((X0_test, X1), axis=0)
y_test = np.concatenate((y0_test, y1), axis=0)

# testar sobre o resto da base
y_pred = predict(p, b, X_test)

# calcular f-score, true positive, true negative
f_score = f1_score(y_test, y_pred, average='weighted')
print('F1-measure: ', f_score)
conf_matrix = confusion_matrix(y_test, y_pred)
true_pos = conf_matrix[1][1]
false_pos = conf_matrix[0][1]
print('True Positives: ', true_pos)
print('False Positives: ', false_pos)

F1-measure:  0.14588782769543326
True Positives:  326
False Positives:  729
